# How to cancel a running workflow

Canceling a workflow in Covalent can be done by invoking its `cancel` API. To cancel, the workflow's dispatch ID is a required argument to the function. Users can optionally also pass in a list of task IDs corresponding to the electrons they wish to cancel from executing from their workflow. Details about the `cancel` API can be found [here](../../api/cancel). For more details regarding this feature and implementation please refer to the documentation [here](../../features/cancel)

Now to demonstrate how a user can cancel a dispatch, let's say we have dispatched a workflow which is going to run for some time. During its execution, we realize that we have given some wrong input and thus want to cancel its execution. That can be done as follows|

In [3]:
import covalent as ct
import time

@ct.electron
def task_1():
    time.sleep(10)
    print("Task 1")
    return 5

@ct.electron
def task_2(b, c):
    time.sleep(b + c)
    print("Task 2")

@ct.lattice
def workflow(x):
    task_2(task_1(), x)
    return x ** 2

dispatch_id = ct.dispatch(workflow)(10)
print(dispatch_id)

6dff7804-5fd8-4081-965d-1a4c7ca94f2a


Now, when we try to get the result of this workflow within next 5 seconds,

In [4]:
time.sleep(3)
result = ct.get_result(dispatch_id)
print(result)


Lattice Result
status: RUNNING
result: None
input args: ['10']
input kwargs: {}
error: None

start_time: 2023-02-27 21:45:47.297742
end_time: None

results_dir: /home/venkat/.local/share/covalent/data
dispatch_id: 6dff7804-5fd8-4081-965d-1a4c7ca94f2a

Node Outputs
------------
task_1(0): None
task_2(1): None
:parameter:10(2): None



We see that it hasn't finished executing yet. At this moment we realize that we want to cancel the execution of this workflow. We use the `ct.cancel` function,

In [5]:
ct.cancel(dispatch_id)

'Dispatch 6dff7804-5fd8-4081-965d-1a4c7ca94f2a cancelled.'

Now we check the result again, and see outputs of all the nodes,

In [6]:
result = ct.get_result(dispatch_id, wait=True)
print(result)

print(result.get_all_node_outputs())


Lattice Result
status: CANCELLED
result: None
input args: ['10']
input kwargs: {}
error: The following tasks failed:


start_time: 2023-02-27 21:45:47.297742
end_time: 2023-02-27 21:45:55.613746

results_dir: /home/venkat/.local/share/covalent/data
dispatch_id: 6dff7804-5fd8-4081-965d-1a4c7ca94f2a

Node Outputs
------------
task_1(0): None
task_2(1): None
:parameter:10(2): 10

{'task_1(0)': None, 'task_2(1)': None, ':parameter:10(2)': <covalent.TransportableObject object at 0x7f2934e89670>}


We see that the after one of the nodes was executed, the cancellation signal was received by the dispatcher and further execution of the workflow was stopped. Since as of yet, `Dask` does not allow cancellation of a running thread, we cannot stop execution of a node that has already started running. So cancellation of a workflow execution will only affect the nodes which hadn't already started their execution and were going to do so after the cancellation signal was received.

### Cancelling a task within a workflow

Let's say now that we want to cancel just `task_2` from the workflow after it has been dispatched. To do so, we simply pass the task's ID to the `ct.cancel` call along with the workflow's dispatch ID. To demonstrate this, lets dispatch the workflow again

In [11]:
dispatch_id = ct.dispatch(workflow)(10)
print(dispatch_id)

3684a5c9-37a2-4834-bef9-7ad96624cd7c


We assert here that the workflow is running properly

In [12]:
time.sleep(3)
result = ct.get_result(dispatch_id)
print(result)


Lattice Result
status: RUNNING
result: None
input args: ['10']
input kwargs: {}
error: None

start_time: 2023-02-27 21:52:20.282375
end_time: None

results_dir: /home/venkat/.local/share/covalent/data
dispatch_id: 3684a5c9-37a2-4834-bef9-7ad96624cd7c

Node Outputs
------------
task_1(0): None
task_2(1): None
:parameter:10(2): None



We see that the workflow is still running. Next we cancel just `task_2` from the workflow as follows

In [13]:
ct.cancel(dispatch_id, task_ids=[1])

'Cancelled tasks [1] in dispatch 3684a5c9-37a2-4834-bef9-7ad96624cd7c.'

Now we check the status of all nodes in the workflow after cancelling `task_2`

In [14]:
result = ct.get_result(dispatch_id, wait=True)
print(result)

print(result.get_all_node_outputs())


Lattice Result
status: CANCELLED
result: None
input args: ['10']
input kwargs: {}
error: The following tasks failed:


start_time: 2023-02-27 21:52:20.282375
end_time: 2023-02-27 21:52:30.380767

results_dir: /home/venkat/.local/share/covalent/data
dispatch_id: 3684a5c9-37a2-4834-bef9-7ad96624cd7c

Node Outputs
------------
task_1(0): 5
task_2(1): None
:parameter:10(2): 10

{'task_1(0)': <covalent.TransportableObject object at 0x7f2934526250>, 'task_2(1)': None, ':parameter:10(2)': <covalent.TransportableObject object at 0x7f2934a27940>}


From the above result we can see that `task_2` as it was cancelled returns `None`